# Method - 3

The huge imbalance in the dataset and very few examples of data makes a Classifier to generalize properly. Instead of framing the problem as a Mutli-Class Classiification problem it will be better to try to frame it as a Regression Problem. The predicted values then can be mapped back to class labels.

The precense of lot of categorical features in the dataset that have a notable significance with the score makes Tree based models an ideal choice. DecisionTreeRegressor is the simples tree based Regressor model that we can implement. In order to make the solution more robust and capture more insights an ensemble method can help. For that a GradientBoostingRegressor can be trained. Finally a Linear Regression model that performed well on the LB can be used.

The predictions from these three models will be weighted averaged and then that prediction can be mapped to class labels or scores.

In [ ]:
# importing libraries

import pandas as pd
import numpy as np
import os

from sklearn.linear_model import LinearRegression, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, mean_absolute_error

In [ ]:
# loading data

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Preprocessing and Feature Engineering

In this section the problems found in the train and test set in EDA will be fixed in this section.

Also, the facility features can be target encoded. It was observed in the EDA that there is a chance for the earthling who stayed in Dec-Feb period to give higher scores. I will call that period winter. Including that information can help the model.


In [ ]:
# fixing the 'hotel_stars' feature in the dataset

train['hotel_stars'] = train['hotel_stars'].map(lambda z : z.replace(',', '.'))
test['hotel_stars'] = test['hotel_stars'].map(lambda z : z.replace(',', '.'))

In [ ]:
# fix mars_membership_years feature in the test data

test['mars_membership_years'] = test['mars_membership_years'].map(lambda y : 0 if y < 0 else y)

In [ ]:
# We have seen in the EDA that features earthling_country and hotel_name have no much significance in the dataset
# Let's drop that from the dataset

del train['earthling_country']
del test['earthling_country']

del train['hotel_name']
del test['hotel_name']

# WE have seen that in the period 'Dec-Feb' there is a higher chance to get higher score
# Let's create a feature stayed_in_winter to indicate whether the earthling stayed in winter or not

train['stayed_in_winter'] = train.period_of_stay.map(lambda x : 1 if x == 'Dec-Feb' else 0)
test['stayed_in_winter'] = test.period_of_stay.map(lambda x : 1 if x == 'Dec-Feb' else 0)

In [ ]:
# Let's label encode the categorical columns for further processing

from sklearn.preprocessing import LabelEncoder, MinMaxScaler

for col in train.columns:
    if train[col].dtype == object:
        enc = LabelEncoder()
        enc.fit(train[col])
        train[col] = enc.transform(train[col])
        test[col] = enc.transform(test[col])

In [ ]:
# Let's scale numerical features in range 0 to 1

num_cols = ['total_reviews', 'hotel_reviews', 'helpful_votes', 'number_of_rooms']

for col in num_cols:
    scaler = MinMaxScaler()
    train[col] = scaler.fit_transform(train[col].values.reshape(-1, 1))
    test[col] = scaler.transform(test[col].values.reshape(-1, 1))

In [ ]:
# target encode the categorical features

for col in ['free_wifi', 'club', 'swimming_pool', 'basketball_court', 'exercise_room', 'yoga_classes',
               'hotel_stars', 'stayed_in_winter']:
    enc_map = train.groupby(by=col)['score'].mean().to_dict()
    train[col+'_encoded'] = train[col].map(enc_map)
    test[col+'_encoded'] = test[col].map(enc_map)

# Model Training

3 models will be trained on the same data and a weighted average will be taken. The parameters are tuned on CV and LB performance.

In [ ]:
# Split data in to train and test

train_feats, valid_feats, train_labels, valid_labels = train_test_split(train.drop(['score'], axis=1), train['score'],
                                                                       test_size=0.10, random_state=2019, stratify=train['score'])

In [ ]:
# Let's frame the problem as a Regression Problem

# We will train 3 regression models
# The model used are GradientBoostingRegressor, DecisionTreeRegressor and LinearRegression
# The data have lot of categorical features, which will help the Tree based DecisionTreeRegressor to find it's way faster
# GradientBoostingRegressor on huber loss is very good on small datasets with outliers and its ensembling technique will help
# LinearRegression model gave the best result yet.

# Let's train all the models and take weighted average and use that to make the final prediction

# mapping function to get labels from the predictions made by the regressor
def map_prediction(score):
    if score < 1.25:
        return 1
    elif score < 2.25:
        return 2
    elif score < 3.25:
        return 3
    elif score < 4.25:
        return 4
    else:
        return 5

# finding best depth for the DecisionTreeRegressor
for d in [2, 3, 4, 5, 6, 7, 8, 9, 10]:
    gbr = GradientBoostingRegressor(n_estimators=100, loss='huber', random_state=20147)
    dtree = DecisionTreeRegressor(max_depth=d, max_leaf_nodes=None, random_state=20147)
    lr = LinearRegression(normalize=False)
    
    gbr.fit(train_feats, train_labels)
    dtree.fit(train_feats, train_labels)
    lr.fit(train_feats, train_labels)
    
    valid_preds = dtree.predict(valid_feats)
    av = gbr.predict(valid_feats)
    lrp = lr.predict(valid_feats)
    
    valid_preds = 0.3 * av + 0.5* valid_preds + 0.2 * lrp

    valid_preds = list(map(map_prediction, valid_preds))
    # print(valid_preds)
    print(f'Depth : {d} => Score : {accuracy_score(valid_labels, valid_preds)}')

Depth 4 seeems best for the DecisionTreeRegressor. 

Let's train the models on full data and then make prediction on the test set

In [ ]:
# preparing test set

test = test[train.drop(['score'], axis=1).columns]

In [ ]:
# training 3 models on complete train data

dtree = DecisionTreeRegressor(max_depth=4, max_leaf_nodes=None, random_state=20147)
dtree.fit(train.drop(['score'], axis=1), train['score'])

gbr = GradientBoostingRegressor(n_estimators=100, loss='huber', random_state=20147)
gbr.fit(train.drop(['score'], axis=1), train['score'])

lr = LinearRegression(normalize=False)
lr.fit(train.drop(['score'], axis=1), train['score'])

In [ ]:
# making prediction

test_preds_gbr = gbr.predict(test)
test_preds_dtree = dtree.predict(test)
test_preds_lr = lr.predict(test)

# taking weighted average
test_preds = 0.3 * test_preds_gbr + 0.5 * test_preds_dtree + 0.2 * test_preds_lr

# mapping predictions to classes
test_preds = list(map(map_prediction, test_preds))

In [ ]:
# loading the sample submission file
sub = pd.read_csv('sample_submission.csv')

In [ ]:
# making submission dataframe

sub['score'] = test_preds

In [ ]:
# saving to local

sub.to_csv('sub_ensemble.csv', index=False)